In [1]:
import observational_fear.load as load
from observational_fear.events import get_freeze_starts
from neurobox.wide_transforms import resample
from neurobox.compose import Pipeline
from observational_fear.nbox_transforms import (
    align_to_data_by,
    exclude_short_trials,
    summarize_prepost,
    wilcoxon_by_cell
)
from observational_fear.stats import auc
from pathlib import Path
from observational_fear.events import get_freeze_stops



DATA_DIR = Path(r"D:\OFL\one-p experiment\data")
FIG_DIR = Path(r"C:\Users\roryl\repos\observational-fear\figs")

In [21]:
load.load_freeze(DATA_DIR, session=SESSION)

,mouse_name,frame,time,was_freezing_dem,was_freezing_obs
0,B17274,1,0.033333,0.0,0.0
1,B17274,2,0.066667,0.0,0.0
2,B17274,3,0.100000,0.0,0.0
3,B17274,4,0.133333,0.0,0.0
4,B17274,5,0.166667,0.0,0.0
...,...,...,...,...,...
518361,PL-OFL-2,64791,2159.700000,0.0,1.0
518362,PL-OFL-2,64792,2159.733333,0.0,1.0
518363,PL-OFL-2,64793,2159.766667,0.0,1.0
518364,PL-OFL-2,64794,2159.800000,0.0,1.0


In [22]:
# load data
SESSION = "day2"
df_freeze = load.load_freeze(DATA_DIR, session=SESSION)
df_cells = load.load_cells(DATA_DIR)
df_cells["new_id"] = df_cells["new_id"].astype("str") 
df_traces = load.load_traces(DATA_DIR, session=SESSION)
df_traces = resample(df_traces.set_index("time").copy(), "100ms")
df_traces = df_traces.reset_index().melt(id_vars=["time"]).merge(df_cells)

In [3]:
detect_freeze_responders = Pipeline(
    [
        (
            align_to_data_by,
            dict(
                df_data_time_col="time",
                df_data_cell_col="new_id",
                df_data_value_col="value",
                time_before_event=5,
                time_after_event=2,
                df_data_group_col="mouse",
                df_events_timestamp_col="freeze_start",
                df_events_group_colname="mouse",
                precision=None,
            ),
        ),
        (
            exclude_short_trials,
            dict(
                trial_col="event",
                min_bins_pre_event=30,
                min_bins_post_event=15,
                cell_col="new_id",
            ),
        ),
        (
            summarize_prepost,
            dict(
                trial_col="event",
                summary_func=auc,
                cell_col="new_id",
                value_col="value",
            ),
        ),
        (
            wilcoxon_by_cell, dict(cell_col="new_id")
        )
    ],
)


In [4]:
# observer
df_events = get_freeze_starts(df_freeze, freeze_col="was_freezing_obs", mouse_col="mouse_name")
df_events = df_events.rename(columns={"mouse_name": "mouse"})
detect_freeze_responders.edit_step_param(index=0, param_name="df_events", param_value=df_events)
res = detect_freeze_responders.transform(df_traces)
df_obs_starts = res.reset_index()

In [5]:
# demonstrator
df_events = get_freeze_starts(df_freeze, freeze_col="was_freezing_dem", mouse_col="mouse_name")
df_events = df_events.rename(columns={"mouse_name": "mouse"})
detect_freeze_responders.edit_step_param(index=0, param_name="df_events", param_value=df_events)
res = detect_freeze_responders.transform(df_traces)
df_dem_starts = res.reset_index()

In [6]:
df_starts = df_obs_starts.merge(df_dem_starts, on="new_id", suffixes=("_obs", "_dem"))
df_starts.to_csv(DATA_DIR / "freeze_starts_day2.csv", index=False)
df_starts.head()

,new_id,statistic_obs,p_obs,diff_of_medians_obs,statistic_dem,p_dem,diff_of_medians_dem
0,0,2011.0,0.053965,1.763946,2546.0,0.849120,0.969757
1,1,1411.0,0.000191,-5.004469,2708.0,0.981522,-1.613901
2,10,2372.0,0.369017,-14.135538,2548.0,0.851357,-5.584139
3,1000,2912.0,0.585363,0.076096,2528.0,0.597718,0.178164
4,1003,3132.0,0.949609,-0.041884,2860.0,0.935030,0.037498


In [7]:
# freeze stops
from observational_fear.events import get_freeze_stops

detect_freeze_responders.edit_step_param(index=0, param_name="df_events_timestamp_col", param_value="freeze_stop")

In [8]:
# observer
df_events = get_freeze_stops(df_freeze, freeze_col="was_freezing_obs", mouse_col="mouse_name")
df_events = df_events.rename(columns={"mouse_name": "mouse"})
detect_freeze_responders.edit_step_param(index=0, param_name="df_events", param_value=df_events)
res = detect_freeze_responders.transform(df_traces)
df_obs_stops = res.reset_index()

In [9]:
# demonstrator
df_events = get_freeze_stops(df_freeze, freeze_col="was_freezing_dem", mouse_col="mouse_name")
df_events = df_events.rename(columns={"mouse_name": "mouse"})
detect_freeze_responders.edit_step_param(index=0, param_name="df_events", param_value=df_events)
res = detect_freeze_responders.transform(df_traces)
df_dem_stops = res.reset_index()

In [10]:
df_stops = df_obs_stops.merge(df_dem_stops, on="new_id", suffixes=("_obs", "_dem"))
df_stops.to_csv(DATA_DIR / "freeze_stops_day2.csv")
df_stops.head()

,new_id,statistic_obs,p_obs,diff_of_medians_obs,statistic_dem,p_dem,diff_of_medians_dem
0,0,2245.0,0.268838,1.094734,2553.0,0.458686,-0.753797
1,1,1564.0,0.001603,4.935058,2800.0,0.714548,0.789340
2,10,1399.0,0.000289,21.385416,3065.0,0.964206,-7.130335
3,1000,2282.0,0.045133,-0.217243,3469.0,0.926681,0.124859
4,1003,3101.0,0.989017,0.258913,3429.0,0.882976,0.034937


In [11]:
# day4

SESSION = "day4"
df_freeze = load.load_behaviour(DATA_DIR, session=SESSION)
df_cells = load.load_cells(DATA_DIR)
df_cells["new_id"] = df_cells["new_id"].astype("str") 
df_traces = load.load_traces(DATA_DIR, session=SESSION)
df_traces = resample(df_traces.set_index("time").copy(), "100ms")
df_traces = df_traces.reset_index().melt(id_vars=["time"]).merge(df_cells)

In [12]:
# get freezes
df_events = get_freeze_starts(df_freeze, freeze_col="was_freezing", mouse_col="mouse_name")
df_events = df_events.rename(columns={"mouse_name": "mouse"})

In [14]:
df_freeze

,frame,time,video_name,fps,x,y,distance,was_freezing,mouse_name
0,1,0.033333,PL-OFL-4_day4_061021_H.265.mp4,30.0,80,51,NaN,0.0,PL-OFL-4
1,2,0.066667,PL-OFL-4_day4_061021_H.265.mp4,30.0,80,51,0.0,0.0,PL-OFL-4
2,3,0.100000,PL-OFL-4_day4_061021_H.265.mp4,30.0,81,51,1.0,0.0,PL-OFL-4
3,4,0.133333,PL-OFL-4_day4_061021_H.265.mp4,30.0,82,51,1.0,0.0,PL-OFL-4
4,5,0.166667,PL-OFL-4_day4_061021_H.265.mp4,30.0,83,52,2.0,0.0,PL-OFL-4
...,...,...,...,...,...,...,...,...,...
64790,64791,2159.700000,OFL_day4_120920_B43396_H.264_TC.mp4,30.0,196,33,0.0,1.0,B43396
64791,64792,2159.733333,OFL_day4_120920_B43396_H.264_TC.mp4,30.0,195,33,1.0,1.0,B43396
64792,64793,2159.766667,OFL_day4_120920_B43396_H.264_TC.mp4,30.0,194,33,1.0,1.0,B43396
64793,64794,2159.800000,OFL_day4_120920_B43396_H.264_TC.mp4,30.0,195,33,1.0,1.0,B43396


In [16]:
# starts
df_events = get_freeze_starts(df_freeze, freeze_col="was_freezing", mouse_col="mouse_name")
df_events = df_events.rename(columns={"mouse_name": "mouse"})
detect_freeze_responders.edit_step_param(index=0, param_name="df_events", param_value=df_events)
detect_freeze_responders.edit_step_param(index=0, param_name="df_events_timestamp_col", param_value="freeze_start")
res = detect_freeze_responders.transform(df_traces)
df_starts = res.reset_index()

In [17]:
# stops
detect_freeze_responders.edit_step_param(index=0, param_name="df_events_timestamp_col", param_value="freeze_stop")

df_events = get_freeze_stops(df_freeze, freeze_col="was_freezing", mouse_col="mouse_name")
df_events = df_events.rename(columns={"mouse_name": "mouse"})
detect_freeze_responders.edit_step_param(index=0, param_name="df_events", param_value=df_events)
res = detect_freeze_responders.transform(df_traces)
df_stops = res.reset_index()

In [23]:
df_starts.to_csv(DATA_DIR / "freeze_starts_day4.csv", index=False)
df_starts.head()

,new_id,statistic,p,diff_of_medians
0,0,1747.0,0.014256,1.453255
1,1,1819.0,0.026252,-3.102287
2,10,2129.0,0.197634,13.006375
3,1003,1700.0,0.423411,0.165341
4,1004,1389.0,0.054430,-7.104406


In [24]:

df_stops.to_csv(DATA_DIR / "freeze_stops_day4.csv")
df_stops.head()

,new_id,statistic,p,diff_of_medians
0,0,3211.0,0.899337,-0.838548
1,1,1399.0,0.000002,5.327043
2,10,3128.0,0.752996,1.523957
3,1003,2359.0,0.556155,-0.131490
4,1004,2308.0,0.460753,2.682057


In [25]:
df_starts

,new_id,statistic,p,diff_of_medians
0,0,1747.0,0.014256,1.453255
1,1,1819.0,0.026252,-3.102287
2,10,2129.0,0.197634,13.006375
3,1003,1700.0,0.423411,0.165341
4,1004,1389.0,0.054430,-7.104406
...,...,...,...,...
1071,989,1454.0,0.092416,1.136096
1072,992,1669.0,0.364165,-0.145983
1073,993,1730.0,0.479633,0.023339
1074,994,1555.0,0.192890,0.008384


In [ ]:
(df_starts.p < 0.05).mean()

0.31784386617100374